In [88]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
cd ~/demres

/Users/zurfarosa/demres


In [90]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [91]:
pd.set_option('display.max_columns', None)

In [92]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

In [93]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

In [94]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )

In [95]:
len(pt_features[pt_features['isCase']==False])

10975

In [96]:
pt_features.sample(5)

,patid,yob,pracid,female,index_date,isCase,final dementia medcode,data_start,data_end,matchid,age_at_index_date,non_insomnia_GP_consultations,insomnia_count,insomnia_count:1-5,insomnia_count:>5,benzo_and_z_drugs:1-1095_pdds,benzo_and_z_drugs:>1096_pdds,benzo_and_z_drugs,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,other_sedatives_100_pdds,fgas_100_pdds,sgas_100_pdds,sga_depots_100_pdds,fga_depots_100_pdds,antidepressants_100_pdds,insomnia,stroke,CHD_heart_failure_and_peripheral_vascular_disease,hypertension,diabetes,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,chronic_pulmonary_disease,epilepsy
17140,7363156,23,156,0,2006-05-30,False,NaN,1996-07-18,2010-02-23,23419,83,41,0,0,0,0,0,0,0.0,0.0,0.346042,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2566,676036,28,36,1,2007-10-10,False,NaN,1999-10-11,2010-10-14,92351,79,36,0,0,0,0,0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
10749,7782158,22,158,0,2006-03-27,False,NaN,1997-10-22,2010-03-05,50474,84,24,0,0,0,0,0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
7636,18148366,9,366,1,2006-04-05,False,NaN,1997-10-01,2009-07-29,23813,97,38,0,0,0,0,0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13132,6298421,26,421,0,2008-07-31,False,NaN,2000-07-18,2013-05-08,37816,82,99,0,0,0,0,0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'female', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'non_insomnia_GP_consultations', 'insomnia_count',
       'insomnia_count:1-5', 'insomnia_count:>5',
       'benzo_and_z_drugs:1-1095_pdds', 'benzo_and_z_drugs:>1096_pdds',
       'benzo_and_z_drugs', 'mood_stabilisers_100_pdds',
       'benzo_and_z_drugs_100_pdds', 'other_sedatives_100_pdds',
       'fgas_100_pdds', 'sgas_100_pdds', 'sga_depots_100_pdds',
       'fga_depots_100_pdds', 'antidepressants_100_pdds', 'insomnia', 'stroke',
       'CHD_heart_failure_and_peripheral_vascular_disease', 'hypertension',
       'diabetes', 'mental_illness_non_smi', 'mental_illness_smi',
       'sleep_apnoea', 'chronic_pulmonary_disease', 'epilepsy'],
      dtype='object')

In [98]:
characteristics = [column for column in pt_features.columns if column not in [
    'patid',  'yob', 'pracid', 'index_date', 'isCase',
    'final dementia medcode', 'data_start', 'data_end', 'matchid']]

In [99]:
baseline_characteristics = pd.DataFrame(index=characteristics,columns=['Cases','Controls','P value'])

In [100]:
baseline_dichot,baseline_contin = add_baseline_characteristics(baseline_characteristics,characteristics,pt_features)

In [101]:
baseline_contin

,Cases - mean,Cases - median,Cases - IQR,Controls - mean,Controls - median,Controls - IQR,P value,test
age_at_index_date,82.26,83.00,78.0 - 87.0,82.26,83.00,78.0 - 87.0,1.000,2-sample-t-test
non_insomnia_GP_consultations,75.22,66.00,38.0 - 100.0,67.65,60.00,33.0 - 91.0,0.000,2-sample-t-test
insomnia_count,0.25,0.00,0.0 - 0.0,0.18,0.00,0.0 - 0.0,0.000,2-sample-t-test
mood_stabilisers_100_pdds,0.22,0.00,0.0 - 0.0,0.11,0.00,0.0 - 0.0,0.000,2-sample-t-test
benzo_and_z_drugs_100_pdds,0.75,0.00,0.0 - 0.0,0.62,0.00,0.0 - 0.0,0.019,2-sample-t-test
other_sedatives_100_pdds,0.09,0.00,0.0 - 0.0,0.09,0.00,0.0 - 0.0,0.727,2-sample-t-test
fgas_100_pdds,0.19,0.00,0.0 - 0.0,0.13,0.00,0.0 - 0.0,0.048,2-sample-t-test
sgas_100_pdds,0.02,0.00,0.0 - 0.0,0.01,0.00,0.0 - 0.0,0.164,2-sample-t-test
sga_depots_100_pdds,0.00,0.00,0.0 - 0.0,0.00,0.00,0.0 - 0.0,nan,2-sample-t-test
fga_depots_100_pdds,0.00,0.00,0.0 - 0.0,0.00,0.00,0.0 - 0.0,0.186,2-sample-t-test


In [102]:
baseline_dichot

,Cases,%Cases,Controls,%Controls,P value,test
female,7030.0,64.1%,7030.0,64.1%,1.000,(matched)
insomnia_count:1-5,1502.0,13.7%,1169.0,10.7%,0.000,chi2
insomnia_count:>5,31.0,0.3%,15.0,0.1%,0.018,chi2
benzo_and_z_drugs:1-1095_pdds,780.0,7.1%,604.0,5.5%,0.000,chi2
benzo_and_z_drugs:>1096_pdds,295.0,2.7%,251.0,2.3%,0.057,chi2
benzo_and_z_drugs,1075.0,9.8%,855.0,7.8%,0.000,chi2
insomnia,1533.0,14.0%,1184.0,10.8%,0.000,chi2
stroke,1037.0,9.4%,775.0,7.1%,0.000,chi2
CHD_heart_failure_and_peripheral_vascular_disease,2755.0,25.1%,2500.0,22.8%,0.000,chi2
hypertension,4523.0,41.2%,4743.0,43.2%,0.003,chi2
